## Data used

lets assume we have table that looks like this 

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns


In [11]:
champ_winrate = {
    'role': ['top', 'top', 'jungle', 'jungle', 'jungle'],
    'champion': ['Maokai', 'Ornn', 'Elise', 'Graves', 'Maokai'],
    'winrate': [54.5, 51.2, 48.3, 52.7, 55.4]
}
df = pd.DataFrame(champ_winrate)

In [12]:
colors = sns.color_palette('rocket', df['champion'].nunique()).as_hex()
color_map = dict(zip(df['champion'].unique(), colors))

In [43]:
from plotly.subplots import make_subplots

def create_role_plots(df, role_order, plot_height=300, plot_width=800):
    fig = make_subplots(
        rows=len(role_order), cols=1,
        subplot_titles=[f"{role.capitalize()}" for role in role_order],
        vertical_spacing=0.06
    )

    for i, role in enumerate(role_order):
        df_role = df[df['role'] == role]
        fig.add_trace(
            go.Bar(
                x=df_role['champion'],
                y=df_role['winrate'],
                marker_color=[color_map[champ] for champ in df_role['champion']],
                name=role
            ),
            row=i+1, col=1
        )
        # Add dashed line at 50% win rate
        fig.add_shape(
            type='line',
            x0=-0.5, x1=len(df_role['champion'].unique())-0.5,
            y0=50, y1=50,
            line=dict(
                color='Black',
                width=2,
                dash='dashdot',
            ),
            row=i+1, col=1
        )

    fig.update_layout(
        title_text='Champion Win Rates by Role',
        height=plot_height * len(role_order),  # Adjust the height based on the number of roles
        width=plot_width,
        plot_bgcolor = '#F5E9DA', 
        paper_bgcolor ='#F5E9DA',
        showlegend=False, 
        font=dict(
            family='Roboto',
            size = 16
        )
    )

    fig.update_yaxes(title_text='Win Rate (%)', range=[0, 100], showgrid = False)
    fig.update_xaxes(title_text='Champion')

    return fig

In [44]:
role_order = ['top', 'jungle', 'mid', 'adc' ,'support']
fig = create_role_plots(df, role_order, plot_height=500, plot_width=1500)
fig.show()